In [1]:
import numpy as np
import scipy as sp
from math import floor
from math import sqrt
from scipy.stats import binom
from scipy.stats import rv_discrete
from tqdm import tqdm

### Subcommitee sampling

In [2]:
# table of stakes (about top100 public + top100 private from tzstats on 2023 07 10)
S = [0.06491, 0.0522, 0.0484, 0.02561, 0.02148, 0.01188, 0.01174, 0.01033, 0.00869, 0.00857, 0.00685, 0.00665, 0.00454, 0.0044, 0.00408, 0.00376, 0.00371, 0.00352, 0.00345, 0.00341, 0.00329, 0.00328, 0.00283, 0.00261, 0.00253, 0.00229, 0.00224, 0.00215, 0.00211, 0.002, 0.00192, 0.00165, 0.00163, 0.00155, 0.00153, 0.00152, 0.00143, 0.00142, 0.00142, 0.00126, 0.00122, 0.00104, 0.00103, 0.00097, 0.00095, 0.00093, 0.00092, 0.00087, 0.00083, 0.00081, 0.00075, 0.00074, 0.00072, 0.0007, 0.00067, 0.00066, 0.00065, 0.00064, 0.00063, 0.00063, 0.00059, 0.00059, 0.00054, 0.00054, 0.00053, 0.00051, 0.00051, 0.00047, 0.00045, 0.00039, 0.00037, 0.00036, 0.00034, 0.00034, 0.00032, 0.00032, 0.0003, 0.0003, 0.0003, 0.0003, 0.00029, 0.00028, 0.00028, 0.00024, 0.00024, 0.00024, 0.00021, 0.00019, 0.00018, 0.00018, 0.00018, 0.00017, 0.00017, 0.00017, 0.00016, 0.00016, 0.00016, 0.00015, 0.00014, 0.00013, 0.00012, 0.00011, 0.00011, 0.00011, 0.00011, 0.0001, 0.0001, 0.0001, 0.00009, 0.00008, 0.00008, 0.00007, 0.00007, 0.00007, 0.00006, 0.00006, 0.00006, 0.00004, 0.00004, 0.00004, 0.00004, 0.00003, 0.00003, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00001, 0.00001, 0.14785, 0.03226, 0.02632, 0.02542, 0.02071, 0.01957, 0.0194, 0.01914, 0.01913, 0.01883, 0.01879, 0.01865, 0.0162, 0.01595, 0.01536, 0.01001, 0.01001, 0.00984, 0.00965, 0.00927, 0.00927, 0.00926, 0.0091, 0.0091, 0.00902, 0.00849, 0.00804, 0.00457, 0.0044, 0.00402, 0.00361, 0.0035, 0.00315, 0.00309, 0.00288, 0.00245, 0.00213, 0.00158, 0.0014, 0.0014, 0.00136, 0.00135, 0.00123, 0.00119, 0.00111, 0.001, 0.00096, 0.00091, 0.00084, 0.00082, 0.00081, 0.00074, 0.00073, 0.00067, 0.00063, 0.00063, 0.00061, 0.00061, 0.00061, 0.00059, 0.00058, 0.00056, 0.00052, 0.0005, 0.00041, 0.0004, 0.0004, 0.00036, 0.00035, 0.00034, 0.00033, 0.00029, 0.00028, 0.00028, 0.00027, 0.00026, 0.00025, 0.00024, 0.00022, 0.00022, 0.00021, 0.0002, 0.00019, 0.00019, 0.00018, 0.00018, 0.00018, 0.00017, 0.00017, 0.00016, 0.00016, 0.00015, 0.00015, 0.00014, 0.00014, 0.00014, 0.00013, 0.00013, 0.00013, 0.00012, 0.00012, 0.00012, 0.00012, 0.00012, 0.00012, 0.00012, 0.00011, 0.00011, 0.00011, 0.00011, 0.00011, 0.00011, 0.0001, 0.00009, 0.00009, 0.00009, 0.00009, 0.00008, 0.00008, 0.00008, 0.00008, 0.00008, 0.00008, 0.00008, 0.00008, 0.00007, 0.00007, 0.00007, 0.00007, 0.00007, 0.00007, 0.00007, 0.00007, 0.00006, 0.00006, 0.00006, 0.00006, 0.00005, 0.00005, 0.00005, 0.00005, 0.00005, 0.00005, 0.00005, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00004, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00003, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00002, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001, 0.00001]
S.sort(reverse=True)
print('5 biggest stakes shares',S[:5])

5 biggest stakes shares [0.14785, 0.06491, 0.0522, 0.0484, 0.03226]


In [3]:
def find_guarantee(eps=1/(100*365.25*24*60*60), p=0.3, shift =1/30, mini = 0, maxi = 10000):
    # find min(n) such that for X = binom(n,p) P(X>=p*n + shift*n) <= eps, and (mini <= n <= maxi)
    if mini == maxi :
        return mini
    n = (mini + maxi) //2
    X= binom(n,p)
    if 1-X.cdf(p*n + shift*n) <= eps:
        return find_guarantee(eps,p, shift, mini, n)
    else:
        return find_guarantee(eps,p, shift, max(n,mini+1), maxi)
     
def find_guarantee_given_S(S,eps=1/(100*365.25*24*60*60), p=0.5, shift =1/30, mini = 0, maxi = 10000):
    # find min(n) such that for X = binom(n,p) P(X>=p*n + shift*n) <= eps, and (mini <= n <= maxi)
    # with the variance reduced distribution, and using the stake distribution S
    if mini == maxi :
        return mini
    n = (mini + maxi) //2
    L = round(n*(1-sum([floor(s*n)/n for s in S])))
    X= binom(L,p)
    if 1-X.cdf(p*L + shift*n) <= eps:
        return find_guarantee_given_S(S,eps,p, shift, mini, n)
    else:
        return find_guarantee_given_S(S,eps,p, shift, max(n,mini+1), maxi)

In [4]:
def estimate_nb_endors(S=S,n=7000,e=10**4) :
#    Given that each agent only sends 1 endorsement message even if he gets several slots 
#    how many message will there be ? <-- this function 
    
#     normalize S and create a random variable
    S_sum=sum(S)
    S_norm=[s/S_sum for s in S]
    X_S = rv_discrete(name='S_distrib', values=(list(range(len(S))), S_norm))
    
#     naïve simulation
    tot=0
    totV=0
    for _ in tqdm(range(e)):
        x = np.unique(X_S.rvs(size = n)).size
        tot+= x
        totV+= x**2
    mean = tot/e
    std = sqrt(totV/e-mean**2)
    return (mean,std)

def estimate_nb_endors_up(S=S,n=870,e=10**4):
#    Given that each agent only sends 1 endorsement message even if he gets several slots 
#    how many message will there be ? <-- this function, but this time with the new method and given S 
    

#     compute number of "free" slots L
    L = round(n*(1-sum([floor(s*n)/n for s in S])))
#     actualize the random distribution then normalize, and create random variable

    S_L = [s%(1/n) for s in S]
    S_sum = sum(S_L)
    S_norm = [s/S_sum for s in S_L]
    X_S = rv_discrete(name='S_distrib', values=(list(range(len(S))), S_norm))
#     number of baker that automatically get >= 1 slot
    n_big = sum([floor(S[i]*n)>=1 for i in range(100)])
    
#     naïve simulation, filtering the slot randomly attributed to a "big" baker
    tot=0
    totV=0
    for _ in tqdm(range(e)):
        x = sum(np.unique(X_S.rvs(size = n))>= n_big)
        tot+= x
        totV+= x**2
    mean = tot/e
    std = sqrt(totV/e-mean**2)
    return (mean,std)

In [5]:
(mean,std) = estimate_nb_endors()
print(f'for the naïve rights distribution, we need {find_guarantee()} of slots')
print(f'meaning with current S, in average there is {mean:.1f} endorsements, with standard deviation {std:.1f}') 
    
(mean,std) =  estimate_nb_endors_up()
print(f'for the improved rights distribution, we need {find_guarantee_given_S(S)} of slots')
print(f'meaning with current S, in average there is {mean:.1f} endorsements, with standard deviation {std:.1f}')

100%|████████████████████████████████████| 10000/10000 [00:21<00:00, 471.83it/s]


for the naïve rights distribution, we need 7353 of slots
meaning with current S, in average there is 230.6 endorsements, with standard deviation 6.5


100%|███████████████████████████████████| 10000/10000 [00:03<00:00, 2535.03it/s]

for the improved rights distribution, we need 870 of slots
meaning with current S, in average there is 155.8 endorsements, with standard deviation 6.1
